In [ ]:
import random
import numpy as np
import torch
import matplotlib.pyplot as plt
from functools import partial

from examples.data.linear_algebra_examples.generator_functions import random_uni_points_in_ball
from src.accuracy_bounds.utils.visualizations import plot_wckersize_conv, plot_avkersize_conv, visualize_ball_3d
from src.accuracy_bounds.inverseproblems.kersize_compute import compute_wc_kernel_size, compute_av_kernel_size
from src.accuracy_bounds.utils.utils import apply_forwardmodel, set_seed
from examples.data.linear_algebra_examples.dataset_generator import GeneratorDataset


### **Toy Example 1:** Worst Case Kernel Size over Uniform Points in 3D Ball 

In [ ]:
num_points = 1000
radius = 2
center = (0,0,0)
dim = 3
epsilon=1e-1
seed = 43

set_seed(seed)


#### Collect Data

In [ ]:
target_data = random_uni_points_in_ball(num_points=num_points, radius=radius+epsilon, center=center, dim=3)                       
visualize_ball_3d(points=target_data, radius=radius, center=center, title="Target Data Points")

### Setup Linear Forward Operator and analytical Kernel Size

In [ ]:
# Toy forward operator
A = np.diag([1, 1, 0])  # Transformation matrix

#analytical wc kersize for ball around 0 with radius r+epsilon
wc_kernel_size_analytical = 2*radius+2*epsilon
#analytical av kersize for ball around 0 with radius r+epsilon sampled with uniform distribution
av_kernel_size_analytical = np.power((radius+epsilon),1/3)

# Distance measure
p=2
q=2

input_data = apply_forwardmodel(A, target_data)

In [ ]:
visualize_ball_3d(points=input_data, radius=radius, center=center, title="Inputs Data Points (Z axis is 0)")

### Setup WC-Kernal Size Computation

In [ ]:
max_k = 300
input_target_pairs = 500

##### Compute Worst-Case Kernel Size

In [ ]:
wc_kersizes = compute_wc_kernel_size(A, input_data[:input_target_pairs], target_data[:input_target_pairs], p, q, epsilon, max_k)
wc_kersize_final = wc_kersizes[-1]

plot_wckersize_conv(wc_kersize_final, wc_kersizes, wc_kernel_size_analytical, max_k)


### Setup AV-Kernal Size Computation

In [ ]:
max_k = 150
input_target_pairs = 1000

In [ ]:

av_kersizes = compute_av_kernel_size(A, input_data[:input_target_pairs], target_data[:input_target_pairs], p, q, epsilon, max_k)

av_kersizef = av_kersizes[-1]

plot_avkersize_conv(av_kersizef, av_kersizes, av_kernel_size_analytical, max_k)